In [15]:
import os
import numpy as np
import pandas as pd
import rasterio
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Activation, Conv2DTranspose, concatenate, UpSampling2D, Concatenate, LayerNormalization, MultiHeadAttention, Dense, Reshape, Multiply, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Layer, Input, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer, Conv2D, BatchNormalization, Activation, UpSampling2D, Add
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K
from tensorflow.keras.utils import Sequence
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import joblib
from scipy.ndimage import rotate
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.metrics import Precision, Recall
import tifffile
import random
import logging
from datetime import datetime
from tensorflow.keras import layers, models
from contextlib import redirect_stdout

# Suppress informational messages and warnings (but not errors)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = all messages are logged, 1 = INFO messages are not printed, 2 = INFO and WARNING messages are not printed, 3 = INFO, WARNING, and ERROR messages are not printed

tf.get_logger().setLevel('ERROR')  # This suppresses TensorFlow INFO and WARNING messages in Python

#데이터 설정
MAX_PIXEL_VALUE = 65535 # 이미지 정규화를 위한 픽셀 최대값
LEARNING_RATE_DECAY = 1


---------------------------------------
<br><br>

> * ### Parameter

In [16]:
# 저장 이름
save_name = 'V4-P'

N_FILTERS = 16 # 필터수 지정
N_CHANNELS = 10 # channel 지정
EPOCHS = 30 # 훈련 epoch 지정
BATCH_SIZE = 64 # batch size 지정
IMAGE_SIZE = (256, 256) # 이미지 크기 지정
MODEL_NAME = 'swinT' # 모델 이름
INITIAL_EPOCH = 0 # 초기 epoch
WORKERS = 64
LEARNING_RATE_DECAY = 0.90
START_LARNING_RATE = 0.01
FINAL_SPARSITY = 0.3

#---------new--------------
AUGMENT = True
#--------------------------

LOSS = 'dice+bc'  # 'dice' , 'focal' , 'tversky' , 'binary_crossentropy' , 'dice+bc' -> 파라미터는 아래서 조정

#---------new--------------
# 모델 파라미터 정의
input_shape = (256, 256, 10)
initial_channels = 64
dim = 64
num_heads = 8
THRESHOLD = 0.5 # 고정값 -> 7번 channel 임계값
#--------------------------

EARLY_STOP_PATIENCE = 5 # 조기종료
CHECKPOINT_PERIOD = 5 # 중간 가중치 저장 이름

# 난수 시드 고정
SEED = 17
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)


# 사용할 데이터의 meta정보 가져오기
train_meta = pd.read_csv('/home/jskim/aispark/AIspark_dataset/train_meta.csv')
test_meta = pd.read_csv('/home/jskim/aispark/AIspark_dataset/test_meta.csv')
# 데이터 위치
IMAGES_PATH = '/home/jskim/aispark/AIspark_dataset/train_img'
MASKS_PATH = '/home/jskim/aispark/AIspark_dataset/train_mask'
# 가중치 저장 위치
OUTPUT_DIR = '/home/jskim/aispark/train_output/'

CHECKPOINT_MODEL_NAME = 'checkpoint-{}-{}-epoch_{{epoch:02d}}.hdf5'.format(MODEL_NAME, save_name)

# 최종 가중치 저장 이름
FINAL_WEIGHTS_OUTPUT = 'model_{}_{}_final_weights.h5'.format(MODEL_NAME, save_name)

# 기본 로그 디렉토리 설정
LOG_DIR = './logs/'

# 로그 파일 이름을 현재 시간으로 설정
log_filename = os.path.join(LOG_DIR, datetime.now().strftime('%Y-%m-%d_%H-%M-%S') + '_' + save_name + '_' + MODEL_NAME + '.log')

# 로깅 기본 설정
logging.basicConfig(filename=log_filename, level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')


# 저장 폴더 없으면 생성
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# 사용할 GPU 이름
CUDA_DEVICE = 1
# GPU 설정
os.environ["CUDA_VISIBLE_DEVICES"] = str(CUDA_DEVICE)
try:
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.compat.v1.Session(config=config)
    K.set_session(sess)
except:
    pass

try:
    np.random.bit_generator = np.random._bit_generator
except:
    pass

2024-03-24 05:10:07.176546: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12591 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:61:00.0, compute capability: 8.9


---------------------------------------
<br><br>

> * ### Encoder structure

In [17]:
class EMSABlock(tf.keras.layers.Layer):
    def __init__(self, dim, num_heads, channel):
        super(EMSABlock, self).__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.channel = channel  # 입력 텐서의 채널 수
        self.depthwise_conv = layers.DepthwiseConv2D(kernel_size=1, padding='same')
        self.layer_norm = layers.LayerNormalization(epsilon=1e-6)

        self.qkv = layers.Dense(dim * 3, use_bias=False)
        self.reshape = layers.Reshape((-1, num_heads, dim // num_heads))
        self.transpose = layers.Permute((2, 1, 3))
        
        self.fc = layers.Dense(self.channel)  # 여기서 self.channel은 입력의 채널 수와 동일해야 합니다.


        self.mlp = models.Sequential([
            layers.Dense(dim * 4, activation='relu'),
            layers.Dense(self.channel),
        ])

    def call(self, inputs):
        # Depthwise Convolution and Layer Norm
        x = self.depthwise_conv(inputs)
        x = self.layer_norm(x)
        
        # Generating q, k, v
        qkv = self.qkv(x)
        q, k, v = tf.split(qkv, 3, axis=-1)
        q = self.transpose(self.reshape(q))
        k = self.transpose(self.reshape(k))
        v = self.transpose(self.reshape(v))
        
        # Attention with MatMul
        attn_output = tf.matmul(q, k, transpose_b=True)
        attn_output = tf.nn.softmax(attn_output)
        attn_output = tf.matmul(attn_output, v)

        # attn_output을 원래 입력 형태로 다시 reshape 합니다.
        batch_size = tf.shape(inputs)[0]
        seq_len = tf.shape(inputs)[1]
        channel = tf.shape(inputs)[3]
        attn_output = tf.reshape(attn_output, [batch_size, seq_len, seq_len, channel])
        
        # Skip connection
        attn_output = self.fc(attn_output) + inputs

        # MLP with Skip connection
        mlp_output = self.mlp(attn_output) + attn_output
        
        return mlp_output

def create_stem_layer(input_shape):
    model = Sequential()
    
    # 첫 번째 Conv2D와 MaxPooling2D를 통해 H/2, W/2의 축소를 수행
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    # 두 번째 Conv2D와 MaxPooling2D를 통해 H/4, W/4의 축소를 수행
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    return model

class PositionalEmbedding2D(tf.keras.layers.Layer):
    def __init__(self, scale=0.1):
        super(PositionalEmbedding2D, self).__init__()
        self.scale = scale

    def build(self, input_shape):
        # `input_shape` 파라미터는 입력 텐서의 형태를 나타냅니다.
        # 이 형태에 기반하여 올바른 차원의 포지셔널 임베딩을 생성합니다.
        height, width, channels = input_shape[1], input_shape[2], input_shape[3]
        self.pos_emb = self.add_weight(
            name="pos_emb", 
            shape=(1, height, width, channels), 
            initializer="random_normal"  # 초기화 방식을 'random_normal'로 변경
        )

    def call(self, inputs):
        # 입력 텐서에 포지셔널 임베딩을 더해 반환합니다.
        return inputs + self.scale * self.pos_emb

def create_downsampling_and_positional_embedding(input_shape, filters):
    model = Sequential()
    # Downsampling
    model.add(Conv2D(filters, kernel_size=(3, 3), strides=(2, 2), padding='same', activation='relu'))
    # Positional Embedding. 여기서는 scale 매개변수만을 사용하면 됩니다.
    # 입력 형태는 build 또는 call 메소드가 호출될 때 자동으로 처리됩니다.
    model.add(PositionalEmbedding2D(scale=0.1))
    return model

def create_e_transformer_block(channels, num_heads):
    # Adjusted to pass `channels`
    e_transformer_block = models.Sequential()
    e_transformer_block.add(EMSABlock(dim, num_heads, channels))
    e_transformer_block.add(EMSABlock(dim, num_heads, channels))
    return e_transformer_block

def create_stage(input_tensor, input_channels, dim, num_heads):
    # This function now takes an input tensor and returns an output tensor.
    stage = tf.keras.Sequential()
    stage.add(PositionalEmbedding2D(input_channels))
    stage.add(create_e_transformer_block(dim, num_heads))
    stage.add(create_e_transformer_block(dim, num_heads))
    return stage(input_tensor)


'''
stage1 : (None, 64, 64, 64)
stage2 : (None, 32, 32, 128)
stage3 : (None, 16, 16, 256)
stage4 : (None, 8, 8, 512)
'''


'\nstage1 : (None, 64, 64, 64)\nstage2 : (None, 32, 32, 128)\nstage3 : (None, 16, 16, 256)\nstage4 : (None, 8, 8, 512)\n'

---------------------------------------
<br><br>

> * ### Decoder structure

In [18]:
def MLPBlock(filters, input_tensor):
    # Define an MLP block to be used in the decoder after each fusion
    mlp = models.Sequential([
        layers.Dense(filters, use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('gelu'),
        layers.Dense(filters, use_bias=False),
        layers.BatchNormalization(),
        layers.Activation('gelu')
    ], name='MLPBlock')
    return mlp(input_tensor)

def FlattenMLPReshape(filters, input_tensor):
    # Flatten, process through an MLP block and reshape for upsampling
    flattened = layers.Flatten()(input_tensor)
    mlp_output = MLPBlock(filters, flattened)
    # Reshape back to a 4D tensor. Note that the shape will be inferred.
    reshaped = layers.Reshape((input_tensor.shape[1] * 2, input_tensor.shape[2] * 2, filters))(mlp_output)
    return reshaped

class PyramidPoolingModule(Layer):
    def __init__(self, bin_sizes, **kwargs):
        super(PyramidPoolingModule, self).__init__(**kwargs)
        self.bin_sizes = bin_sizes

    def build(self, input_shape):
        self.convs = []
        for bin_size in self.bin_sizes:
            self.convs.append(Sequential([
                Conv2D(input_shape[-1], kernel_size=(1, 1), strides=(1, 1), padding='same'),
                BatchNormalization(),
                Activation('relu')
            ]))

    def call(self, inputs):
        concat_list = [inputs]
        h = inputs.shape[1]
        w = inputs.shape[2]
        for bin_size, conv in zip(self.bin_sizes, self.convs):
            x = tf.keras.layers.AveragePooling2D(pool_size=(h // bin_size, w // bin_size))(inputs)
            x = conv(x)
            x = UpSampling2D(size=(h // x.shape[1], w // x.shape[2]))(x)
            concat_list.append(x)
        return Concatenate()(concat_list)

def create_decoder(C1, C2, C3, C4):
    # PPM을 사용한 후 채널 수 조정을 위해 1x1 컨볼루션 레이어를 적용
    ppm = PyramidPoolingModule(bin_sizes=[1, 2, 3, 6])(C4)
    ppm = Conv2D(256, kernel_size=(1, 1), padding='same')(ppm)  # C3와 채널 수 일치
    upsampled_ppm = UpSampling2D(size=(2, 2))(ppm)
    
    # C3와 결합
    P3 = Add()([upsampled_ppm, C3])
    upsampled_P3 = UpSampling2D(size=(2, 2))(P3)
    
    # C2와 결합하기 전에 C2의 채널 수를 맞춥니다.
    C2_adjusted = Conv2D(256, kernel_size=(1, 1), padding='same')(C2)
    P2 = Add()([upsampled_P3, C2_adjusted])
    upsampled_P2 = UpSampling2D(size=(2, 2))(P2)
    
    # C1과 결합하기 전에 C1의 채널 수를 맞춥니다.
    C1_adjusted = Conv2D(256, kernel_size=(1, 1), padding='same')(C1)
    P1 = Add()([upsampled_P2, C1_adjusted])

    # 컨볼루션을 적용하여 최종 세그멘테이션 맵 생성
    segmentation_map = Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')(P1)

    return segmentation_map





---------------------------------------
<br><br>

> * ### dataset , metric

In [19]:
def edge_enhancement(original_image, segmentation_result):
    # 원본 이미지에서 7번째 채널을 선택합니다.
    channel_7 = original_image[..., 6:7]  # 0-indexed, 7번째 채널은 인덱스 6입니다.
    # 임계값 이상의 픽셀을 선별하고, 나머지 픽셀을 0으로 설정합니다.
    highlighted_pixels = tf.where(channel_7 >= THRESHOLD, channel_7, 0)

    # 선별된 픽셀 값을 최대 픽셀 값으로 확장합니다.
    # 선별된 픽셀들에 대해 (픽셀 값 - 임계값)을 계산하여 범위를 [0, MAX_PIXEL_VALUE]로 매핑합니다.
    highlighted_pixels_scaled = tf.where(highlighted_pixels > 0, ((highlighted_pixels - THRESHOLD) / (MAX_PIXEL_VALUE - THRESHOLD)) * MAX_PIXEL_VALUE, 0)

    # 엣지 추출
    edge_features = Conv2D(1, (4, 4), padding='same', activation='sigmoid')(highlighted_pixels_scaled)

    # 이진화
    def binarize(x):
        return K.cast(K.greater(x, 0.5), K.floatx())  # 임계값을 0.5로 설정

    binarized_edges = Lambda(binarize)(edge_features)

    # 정규화
    normalized_edges = BatchNormalization()(binarized_edges)

    # 정규화된 엣지 맵을 세그멘테이션 결과에 더합니다.
    segmentation_result_upsampled = UpSampling2D(size=(4, 4))(segmentation_result)  # 64x64에서 256x256으로 업샘플링
    enhanced_segmentation = Add()([segmentation_result_upsampled, normalized_edges])

    return enhanced_segmentation

def create_final_model(input_shape, initial_channels, dim, num_heads):
    # 입력 텐서를 정의합니다.
    input_tensor = Input(shape=input_shape)
    
    # Stem and downsampling layers
    x = create_stem_layer(input_shape)(input_tensor)
    
    # Stages
    # We create the stages by calling them with the output tensor from the previous layer.
    stage1_output = create_stage(x, initial_channels, dim, num_heads)
    DS_PE2_output = create_downsampling_and_positional_embedding((input_shape[0]//4, input_shape[1]//4), initial_channels*2)(stage1_output)
    stage2_output = create_stage(DS_PE2_output, initial_channels*2, dim*2, num_heads*2)
    DS_PE3_output = create_downsampling_and_positional_embedding((input_shape[0]//8, input_shape[1]//8), initial_channels*4)(stage2_output)
    stage3_output = create_stage(DS_PE3_output, initial_channels*4, dim*4, num_heads*4)
    DS_PE4_output = create_downsampling_and_positional_embedding((input_shape[0]//16, input_shape[1]//16), initial_channels*8)(stage3_output)
    stage4_output = create_stage(DS_PE4_output, initial_channels*8, dim*8, num_heads*8)
        
    # Decoder to get the segmentation map
    segmentation_result = create_decoder(stage1_output, stage2_output, stage3_output, stage4_output)
    
    # Edge enhancement
    enhanced_segmentation_output = edge_enhancement(input_tensor, segmentation_result)
    
    # Final model
    final_model = Model(inputs=input_tensor, outputs=enhanced_segmentation_output)
    
    return final_model

In [20]:
class DataGenerator(Sequence):
    def __init__(self, img_paths, mask_paths, batch_size=32, default_path=None, augment=False, add_noise=False, noise_factor=0.01, shuffle=True):
        self.img_paths = img_paths
        self.mask_paths = mask_paths
        self.batch_size = batch_size
        self.default_path = default_path if default_path else ""
        self.augment = augment
        self.add_noise = add_noise
        self.noise_factor = noise_factor
        self.shuffle = shuffle

        if self.shuffle:
            self.on_epoch_end()

    def __len__(self):
        return len(self.img_paths) // self.batch_size

    def __getitem__(self, idx):
        batch_img_paths = self.img_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_mask_paths = self.mask_paths[idx * self.batch_size:(idx + 1) * self.batch_size]

        batch_imgs = []
        batch_masks = []

        for img_path, mask_path in zip(batch_img_paths, batch_mask_paths):
            img = tifffile.imread(os.path.join(self.default_path, img_path))
            mask = tifffile.imread(os.path.join(self.default_path, mask_path))

            # 이미지 전처리
            if self.augment:
                img, mask = self.augment_image(img, mask)

            img = img.astype(np.float32) / MAX_PIXEL_VALUE
            mask = mask.astype(np.float32)
            mask = np.expand_dims(mask, axis=-1)

            batch_imgs.append(img)
            batch_masks.append(mask)
        return np.array(batch_imgs), np.array(batch_masks)

    def on_epoch_end(self):
        if self.shuffle:
            indices = np.arange(len(self.img_paths))
            np.random.shuffle(indices)
            self.img_paths = np.array(self.img_paths)[indices].tolist()
            self.mask_paths = np.array(self.mask_paths)[indices].tolist()

    def augment_image(self, img, mask):
        decision_flip_lr = tf.random.uniform([], seed=SEED) < 0.25
        decision_flip_ud = tf.random.uniform([], seed=SEED) > 0.75
        decision_noise = tf.random.uniform([], seed=SEED) < 0.1

        if decision_flip_lr:
            img = np.flip(img, axis=1)
            mask = np.flip(mask, axis=1)

        if decision_flip_ud:
            img = np.flip(img, axis=0)
            mask = np.flip(mask, axis=0)

        # 노이즈 추가 부분도 동일한 방식으로 랜덤 결정을 시드와 함께 사용
        if self.add_noise and decision_noise:
            img = self.add_gaussian_noise(img, self.noise_factor)

        return img, mask

    
    def shuffle_lists(self, images_path, masks_path, random_state=None):
        # random_state가 주어진 경우, 난수 생성기의 시드를 설정합니다.
        if random_state is not None:
            random.seed(random_state)
        
        # images_path와 masks_path 리스트를 함께 섞기 위해 두 리스트의 쌍을 만듭니다.
        paired_lists = list(zip(images_path, masks_path))
        random.shuffle(paired_lists)
        
        # 섞인 리스트를 다시 분리합니다.
        shuffled_images_path, shuffled_masks_path = zip(*paired_lists)
        
        # zip() 함수는 튜플을 반환하므로, 리스트로 변환해 반환합니다.
        return list(shuffled_images_path), list(shuffled_masks_path)


    def add_gaussian_noise(self, image, mean=0, std=0.01, noise_factor=0.1):
        """
        이미지에 가우시안 노이즈를 추가하는 함수.
        mean: 노이즈의 평균값
        std: 노이즈의 표준편차
        noise_factor: 노이즈의 전체적인 강도를 조절하는 계수
        """
        gaussian_noise = np.random.normal(mean, std, image.shape) * noise_factor * MAX_PIXEL_VALUE
        noisy_image = image + gaussian_noise
        noisy_image = np.clip(noisy_image, 0, MAX_PIXEL_VALUE)  # 픽셀 값을 0과 MAX_PIXEL_VALUE 사이로 제한
        return noisy_image
    
def lr_decay(epoch, lr):
    """
    에포크에 따라 학습률을 감소시키는 함수.
    매 epoch마다 학습률을 0.95배 해주는 예시.
    """
    new_lr = lr * LEARNING_RATE_DECAY
    logging.info(f"Learning rate : {new_lr}")
    return new_lr

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

class IoU(tf.keras.metrics.Metric):
    def __init__(self, name='iou', **kwargs):
        super(IoU, self).__init__(name=name, **kwargs)
        self.intersection = self.add_weight(name="intersection", initializer="zeros")
        self.union = self.add_weight(name="union", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.round(y_pred)  # 세그멘테이션 마스크를 이진 형태로 변환
        intersection = tf.reduce_sum(y_true * y_pred)
        union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
        
        # 이전 상태에 누적
        self.intersection.assign_add(intersection)
        self.union.assign_add(union)

    def result(self):
        return self.intersection / (self.union + tf.keras.backend.epsilon())

    def reset_states(self):
        # 에포크가 끝날 때마다 상태 초기화
        self.intersection.assign(0)
        self.union.assign(0)

class LoggingCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        # IoU 값을 로그에 포함시키기 위해 'iou'와 'val_iou' 키를 로그에서 검색
        # 키가 없는 경우 None을 반환하기 위해 get 메소드 사용
        iou = logs.get('iou')
        val_iou = logs.get('val_iou')
        # IoU 값이 제공되는 경우 로그 메시지에 포함, 그렇지 않은 경우 "N/A"로 표시
        logging.info(f"Epoch {epoch + 1}, Loss: {logs['loss']}, Accuracy: {logs['accuracy']}, Val_Loss: {logs['val_loss']}, Val_Accuracy: {logs['val_accuracy']}, IoU: {iou if iou is not None else 'N/A'}, Val_IoU: {val_iou if val_iou is not None else 'N/A'}")

def dice_loss_function(y_true, y_pred):
    smooth = 1e-6
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return 1 - (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_coef(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return (numerator + 1) / (denominator + 1)

def combined_dice_crossentropy_loss(y_true, y_pred):
    dice_loss = dice_loss_function(y_true, y_pred)  # 앞서 정의한 dice_loss_function 사용
    cross_entropy_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return dice_loss + cross_entropy_loss

def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.sum((1-alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))
    return focal_loss_fixed

def tversky_loss(beta):
    def loss(y_true, y_pred):
        numerator = K.sum(y_true * y_pred, axis=-1)
        denominator = y_true * y_pred + beta * (1 - y_true) * y_pred + (1 - beta) * y_true * (1 - y_pred)
        return 1 - (numerator + 1) / (K.sum(denominator, axis=-1) + 1)
    return loss

# 두 샘플 간의 유사성 metric
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
    return dice

# 이미지 경로에서 이미지를 읽어와 정규화한 후 numpy 배열로 반환
def get_img_arr(path):
    img = rasterio.open(path).read().transpose((1, 2, 0))
    img = np.float32(img)/MAX_PIXEL_VALUE

    return img

# 마스크 이미지 경로에서 이미지를 읽어와 numpy 배열로 반환
def get_mask_arr(path):
    img = rasterio.open(path).read().transpose((1, 2, 0))
    seg = np.float32(img)
    return seg


---------------------------------------
<br><br>

> * ### Training

In [21]:
#------------------------new------------------------
ADD_NOISE = False
NOISE_FACTOR=0.1
LOSS=combined_dice_crossentropy_loss,
METRICS = [IoU(), 'accuracy', F1Score(), dice_coef]
#---------------------------------------------------


# 변수 로그에 기록
logging.info(f'Save name: {save_name}')
logging.info(f'Number of filters: {N_FILTERS}')
logging.info(f'Number of channels: {N_CHANNELS}')
logging.info(f'Epochs: {EPOCHS}')
logging.info(f'Batch size: {BATCH_SIZE}')
logging.info(f'Image size: {IMAGE_SIZE}')
logging.info(f'Model name: {MODEL_NAME}')
logging.info(f'Initial epoch: {INITIAL_EPOCH}')
logging.info(f'Workers: {WORKERS}')
logging.info(f'Learning rate decay: {LEARNING_RATE_DECAY}')
logging.info(f'Start learning rate: {START_LARNING_RATE}')
logging.info(f'Seed: {SEED}')

# train : val = 8 : 2 나누기
x_tr, x_val = train_test_split(train_meta, test_size=0.2, random_state=SEED)
print(len(x_tr), len(x_val))

# train : val 지정 및 generator
images_train = [os.path.join(IMAGES_PATH, image) for image in x_tr['train_img'] ]
masks_train = [os.path.join(MASKS_PATH, mask) for mask in x_tr['train_mask'] ]

images_validation = [os.path.join(IMAGES_PATH, image) for image in x_val['train_img'] ]
masks_validation = [os.path.join(MASKS_PATH, mask) for mask in x_val['train_mask'] ]

# 학습 데이터용 인스턴스 생성, 데이터 증강 적용
train_generator = DataGenerator(images_train, masks_train, batch_size=BATCH_SIZE, default_path=IMAGES_PATH, augment=AUGMENT,add_noise=ADD_NOISE, noise_factor=NOISE_FACTOR)
# 검증 데이터용 인스턴스 생성, 데이터 증강 미적용
validation_generator = DataGenerator(images_validation, masks_validation, batch_size=BATCH_SIZE, default_path=IMAGES_PATH, augment=False, add_noise=False)

# 최종 모델 생성
final_model = create_final_model(input_shape, initial_channels, dim, num_heads)
# 모델 컴파일
final_model.compile(optimizer=Adam(learning_rate=START_LARNING_RATE),
                    loss=LOSS,
                    metrics=METRICS)

# 콜백 정의
es = EarlyStopping(monitor='val_iou', mode='max', verbose=1, patience=EARLY_STOP_PATIENCE)
checkpoint = ModelCheckpoint(os.path.join(OUTPUT_DIR, CHECKPOINT_MODEL_NAME), 
                             monitor='val_iou',
                             verbose=1, save_best_only=True, mode='max', save_freq='epoch')
lr_scheduler = LearningRateScheduler(lr_decay)

# 콜백에 Pruning 콜백 추가
callbacks = [
    checkpoint,
    es,
    lr_scheduler,
    LoggingCallback()
]

# 모델 훈련
history = final_model.fit(
    train_generator,
    steps_per_epoch=len(images_train) // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=len(images_validation) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[checkpoint, es, lr_scheduler, LoggingCallback()],
    workers=WORKERS,
    initial_epoch=INITIAL_EPOCH
)

# 필요 시 final_model 저장
logging.info('가중치 저장')
model_weights_output = os.path.join(OUTPUT_DIR, FINAL_WEIGHTS_OUTPUT)
final_model.save(model_weights_output)
logging.info(f"저장된 가중치 명: {model_weights_output}")



26860 6715


Epoch 1/30


2024-03-24 05:10:24.332803: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:231] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-03-24 05:10:24.332835: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Used ptxas at ptxas
2024-03-24 05:10:24.332935: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-03-24 05:10:24.364014: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8800
2024-03-24 05:10:24.427978: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-03-24 05:10:24.429204: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by dri

RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL ResourceExhaustedError REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /home/jskim/anaconda3/envs/aispark/lib/python3.9/site-packages/tensorflow/python/framework/errors_impl.py(377): __init__
  /home/jskim/anaconda3/envs/aispark/lib/python3.9/site-packages/tensorflow/python/eager/execute.py(60): quick_execute
  /home/jskim/anaconda3/envs/aispark/lib/python3.9/site-packages/tensorflow/python/eager/context.py(1479): call_function
  /home/jskim/anaconda3/envs/aispark/lib/python3.9/site-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py(252): __call__
  /home/jskim/anaconda3/envs/aispark/lib/python3.9/site-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py(217): flat_call
  /home/jskim/anaconda3/envs/aispark/lib/python3.9/site-packages/tensorflow/python/eager/polymorphic_function/concrete_function.py(1264): _call_flat
  /home/jski

---------------------------------------
<br><br>

> * ### Make Pickle , Submit format

In [ ]:
# 제출용 pkl 파일 만들기
final_model = create_final_model(input_shape, initial_channels, dim, num_heads)
final_model.compile(optimizer = Adam(), loss = LOSS, metrics = ['accuracy'])
final_model.summary()

final_model.load_weights('model_{}_{}_final_weights.h5'.format(MODEL_NAME, save_name))

y_pred_dict = {}

# 가정: test_meta는 테스트 이미지 파일명을 포함하는 딕셔너리나 리스트입니다.
for i in test_meta['test_img']:  # tqdm은 진행 상태 바를 표시합니다.
    img = get_img_arr(f'/home/jskim/aispark/AIspark_dataset/test_img/{i}')
    y_pred = model.predict(np.array([img]), batch_size=1)

    y_pred = np.where(y_pred[0, :, :, 0] > 0.25, 1, 0)  # 임계값 처리
    y_pred = y_pred.astype(np.uint8)
    y_pred_dict[i] = y_pred

joblib.dump(y_pred_dict, '/home/jskim/aispark/y_pred_{}_{}.pkl'.format(MODEL_NAME, save_name))


"\n    optimizer = torch.optim.SGD([{'params':\n                                      filter(lambda p: p.requires_grad,\n                                             model.parameters()),\n                                      'lr': args2.lr}],\n                                    lr=args2.lr,\n                                    momentum=0.9,\n                                    weight_decay=0.0005,\n                                    nesterov=False,\n                                    )\n\n    optimizer = torch.optim.AdamW([{'params':\n                                      filter(lambda p: p.requires_grad,\n                                             model.parameters()),\n                                  'lr': args2.lr}],\n                                lr=args2.lr,\n                                betas=(0.9, 0.999),\n                                weight_decay=0.01,\n                                )\n"

---------------------------------------
<br><br>

> * ### Visualization

In [ ]:
# 특정 이미지에 대한 예측 마스크 시각화 함수
def visualize_prediction(test_image_name):
    # 테스트 이미지 로딩
    img = get_img_arr(f'./AIspark_dataset/test_img/{test_image_name}')
    
    # 예측된 마스크 로딩
    y_pred = y_pred_dict[test_image_name]
    
    # 이미지와 예측된 마스크 시각화
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(img[:, :, 0], cmap='gray')  # 첫 번째 채널을 그레이스케일 이미지로 시각화
    axs[0].set_title('Original Image')
    axs[0].axis('off')

    axs[1].imshow(y_pred, cmap='gray')  # 예측된 마스크 시각화
    axs[1].set_title('Predicted Mask')
    axs[1].axis('off')

    plt.show()

def load_tif_image(image_path):
    with rasterio.open(image_path) as img_file:
        img = img_file.read()
    return np.transpose(img, (1, 2, 0))

def visualize_channel_images(image):
    channels = image.shape[2]
    plt.figure(figsize=(20, 10))
    for i in range(channels):
        plt.subplot(2, (channels+1)//2, i+1)
        plt.imshow(image[:, :, i])
        plt.title(f'Channel {i+1}')
        plt.axis('off')
    plt.show()

# 시각화
# 예측 결과 딕셔너리 로드
y_pred_dict = joblib.load('/home/jskim/aispark/y_pred_{}_{}.pkl'.format(MODEL_NAME, save_name))

# 예시: 'test_img_2427.tif' 이미지에 대한 예측 마스크 시각화
visualize_prediction('test_img_2423.tif')
image_path = '/home/jskim/aispark/AIspark_dataset/test_img/test_img_2423.tif'
visualize_channel_images(load_tif_image(image_path))
    